In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Retrieval Augmented Generation (RAG) multimodal usando a Vertex AI Gemini API

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/retrieval-augmented-generation/intro_multimodal_rag.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Execute no Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/retrieval-augmented-generation/intro_multimodal_rag.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> Veja no GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/retrieval-augmented-generation/intro_multimodal_rag.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Abra no Workbench da Vertex AI
    </a>
  </td>
</table>


## Visão geral

A geração aumentada de recuperação (RAG) é uma abordagem popular para permitir que LLMs acessem dados externos e também como um mecanismo de *groundin* para mitigar alucinações.

Neste notebook, você aprenderá como realizar RAG multimodal, onde realizará perguntas e respostas sobre um documento financeiro preenchido com texto e imagens.

### Gemini

Gemini é uma família de modelos generativos de IA desenvolvidos pelo Google DeepMind e projetados para casos de uso multimodais. A API Gemini dá acesso aos modelos `Gemini Pro Vision` e `Gemini Pro`.

### Comparando RAG baseado em texto e multimodal

O RAG multimodal oferece diversas vantagens sobre o RAG baseado em texto:

1. **Acesso aprimorado ao conhecimento:** O RAG multimodal pode acessar e processar informações textuais e visuais, fornecendo uma base de conhecimento mais rica e abrangente para o LLM.
2. **Capacidades de raciocínio aprimoradas:** Ao incorporar dicas visuais, o RAG multimodal pode fazer inferências mais bem informadas em diferentes tipos de modalidades de dados.

Este notebook mostra como usar o RAG com a API Vertex AI Gemini, [embeddings de texto](https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/text-embeddings) e [ embeddings multimodais](https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/multimodal-embeddings), para criar um mecanismo de pesquisa de documentos.

Através de exemplos práticos, você descobrirá como construir um repositório de metadados rico em multimídia de suas fontes de documentos, permitindo pesquisa, comparação e raciocínio em diversos fluxos de informações.

### Objetivos

Este notebook fornece um guia para construir um mecanismo de busca de documentos usando geração aumentada de recuperação multimodal (RAG), passo a passo:

1. Extraia e armazene metadados de documentos contendo texto e imagens e gere incorporações nos documentos
2. Pesquise os metadados com consultas de texto para encontrar textos ou imagens semelhantes
3. Pesquise os metadados com consultas de imagens para encontrar imagens semelhantes
4. Usando uma consulta de texto como entrada, pesquise respostas contextuais usando texto e imagens

### Custos

Este tutorial usa os seguintes componentes de Google Cloud que podem gerar custos em sua fatura:

- Vertex AI

Saiba mais sobre [preços da Vertex AI](https://cloud.google.com/vertex-ai/pricing) e use a [calculadora de preços](https://cloud.google.com/products/calculator/) para gerar uma estimativa de custo com base no uso projetado.

## Primeiros passos

### Instale a SDK da Vertex AI
**Importante:** só descomente a linha abaixo se você **não estiver** executando este laboratório no Qwiklabs

In [ ]:
# ! pip3 install --upgrade --user google-cloud-aiplatform pymupdf

### **Reinicie o kernel do seu jupyter notebook** 

Como a instalação está sendo realizada com a opção `--user` é necessário reiniciar o kernel para que os novos módulos se tornem acessíveis.

**Importante:** só descomente a linha abaixo se você **não estiver** executando este laboratório no Qwiklabs

In [ ]:
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ O kernel do notebook está sendo reiniciado. Por favor aguarde este processo ser finalizado antes de continuar com os próximos passos. ⚠️</b>
</div>

### **Somente para uso no Colab - Autentique o seu ambiente de notebook** 

Caso você esteja executando este notebook no Google Colab, descomente a célula abaixo para realizar a autenticação da sua sessão de notebook com a Google Cloud Esse passo é importante **para utilização no Colab** para garantir que as chamadas a APIs de Google Cloud funcionem sem problemas.

In [ ]:
# import sys

# # Additional authentication is required for Google Colab
# if "google.colab" in sys.modules:
#     # Authenticate user to Google Cloud
#     from google.colab import auth

#     auth.authenticate_user()

### **Somente para uso no Colab - defina o projeto Google Cloud a ser utilizado** 

Caso você esteja executando este notebook no Google Colab, descomente a célula abaixo para definir qual projeto Google Cloud será utilizado pelo Colab na execução deste notebook. Senão, siga para as próximas instruções.

In [ ]:
# if "google.colab" in sys.modules:
#     # Define project information
#     PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
#     LOCATION = "us-central1"  # @param {type:"string"}

#     # Initialize Vertex AI
#     import vertexai

#     vertexai.init(project=PROJECT_ID, location=LOCATION)

### Importe as bibliotecas necessárias

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import warnings
warnings.simplefilter("ignore", UserWarning)

from IPython.display import Markdown, display
from vertexai.generative_models import (
    Content,
    GenerationConfig,
    GenerationResponse,
    GenerativeModel,
    Image,
    Part,
)

PROJECT_ID = "lucianomartins-demos-345000"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

## Importe o modelo `Gemini 1.0 Pro`

O Gemini Pro (`gemini-1.0-pro`) ajuda na realização de tarefas utilizando linguagem natural, chats multiturno de texto e código e para a geração de código.

In [ ]:
model = GenerativeModel("gemini-1.0-pro-vision")

### Baixe módulos e utilitários Python customizados

A célula abaixo fará o download de algumas funções auxiliares necessárias para este notebook. Você também pode visualizar o código (`intro_multimodal_rag_utils.py`) diretamente no [Github](https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/use-cases/retrieval-augmented-generation/utils/intro_multimodal_rag_utils.py).

In [ ]:
import os
import urllib.request
import sys

if not os.path.exists("utils"):
    os.makedirs("utils")

    
# download the helper scripts from utils folder
url_prefix = "https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/use-cases/retrieval-augmented-generation/utils/"
files = ["intro_multimodal_rag_utils.py"]

for fname in files:
    urllib.request.urlretrieve(f"{url_prefix}/{fname}", filename=f"utils/{fname}")

#### Baixe documentos e imagens do Cloud Storage

In [ ]:
# download documents and images used in this notebook
!gsutil -m rsync -r gs://github-repo/rag/intro_multimodal_rag .
print("Download finalizado.")

## Construa os metadados dos documentos (contendo texto e imagem)

### Os dados

Os dados que você usará neste notebook são uma versão modificada do [Google-10K](https://abc.xyz/assets/investor/static/pdf/20220202_alphabet_10K.pdf) que fornece uma visão geral abrangente da situação financeira da Alphabet, como desempenho, operações de negócios, gerenciamento e fatores de risco. Como o documento original é bastante grande, você usará [uma versão modificada com apenas 14 páginas](https://storage.googleapis.com/github-repo/rag/intro_multimodal_rag/google-10k-sample-14pages.pdf). Embora esteja truncado, o documento de amostra ainda contém texto junto com imagens como tabelas, tabelas e gráficos.

### Importe funções auxiliares para construir metadados

Antes de construir o sistema RAG multimodal, é importante ter metadados de todos os textos e imagens do documento. Para fins de referências e citações, os metadados devem conter elementos essenciais, incluindo número de página, nome do arquivo, contador de imagens e assim por diante. Portanto, como próxima etapa, você gerará embeddings a partir dos metadados, que serão necessários para realizar a pesquisa de similaridade ao consultar os dados.

In [ ]:
from utils.intro_multimodal_rag_utils import get_document_metadata

### Extraia e armazene metadados de texto e imagens de um documento

Você acabou de importar uma função chamada `get_document_metadata()`. Esta função extrai metadados de texto e imagem de um documento e retorna dois dataframes, *text_metadata* e *image_metadata*, como saídas. Se você quiser saber mais sobre como a função `get_document_metadata()` é implementada usando Gemini e os modelos de *embeddings*, você pode dar uma olhada no [código-fonte](https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/use-cases/retrieval-augmented-generation/utils/intro_multimodal_rag_utils.py) diretamente.

A razão para extrair e armazenar metadados de texto e metadados de imagem é que apenas usar um dos dois não é suficiente para obter uma resposta relevante. Por exemplo, as respostas relevantes poderiam estar em formato visual dentro de um documento, mas o RAG baseado em texto não poderá levar em consideração as imagens visuais. Você também explorará esse exemplo posteriormente neste notebook.

Na próxima etapa, você usará a função para extrair e armazenar metadados de texto e imagens de um documento. Observe que a célula a seguir pode levar alguns minutos para ser concluída:

In [ ]:
pdf_folder_path = "data/"

image_description_prompt = """Explique a imagem.
Não inclua números que não sejam mencionados na imagem:"""

# Se for uma tabela, extraia os elementos da tabela.
# Se for um gráfico, explique as descobertas no gráfico.

text_metadata_df, image_metadata_df = get_document_metadata(
    model,
    pdf_folder_path,
    image_save_dir="images",
    image_description_prompt=image_description_prompt,
    embedding_size=1408
)

print("--- Processamento finalizado. ---")

#### Inspecione os metadados de texto processados

A célula a seguir produzirá uma tabela de metadados que descreve as diferentes partes dos metadados de texto, incluindo:

- **text**: o texto original da página
- **text_embedding_page**: o *embedding* do texto original da página
- **chunk_text**: o texto original dividido em pedaços menores
- **chuck_number**: o índice de cada pedaço de texto
- **text_embedding_chunk**: a *embedding* de cada pedaço de texto

In [ ]:
text_metadata_df.head()

#### Inspecione os metadados das imagens processadas

A célula a seguir produzirá uma tabela de metadados que descreve as diferentes partes dos metadados da imagem, incluindo:
* **img_desc**: descrição textual da imagem gerada pelo Gemini.
* **mm_embedding_from_text_desc_and_img**: Embedding combinada de imagem e sua descrição, capturando informações visuais e textuais.
* **mm_embedding_from_img_only**: Embedding de imagem sem descrição, para comparação com análise baseada em descrição.
* **text_embedding_from_image_description**: Embedding de texto separada da descrição gerada, permitindo análise e comparação textual.

In [ ]:
image_metadata_df.head()

### Importe as funções auxiliares para implementar o RAG

Você importará as seguintes funções que serão usadas no restante deste notebook para implementar o RAG:

* **get_similar_text_from_query():** Dada uma consulta de texto, encontra textos do documento que são relevantes, usando o algoritmo de similaridade de cosseno. Ele usa incorporações de texto dos metadados para calcular e os resultados podem ser filtrados por pontuação máxima, número de página/bloco ou tamanho de incorporação.
* **print_text_to_text_citation():** Cite a fonte (citação) e detalhes do texto recuperado da função `get_similar_text_from_query()`.
* **get_similar_image_from_query():** Dado um caminho de imagem ou uma imagem, encontra imagens do documento que são relevantes. Ele usa incorporações de imagens dos metadados.
* **print_text_to_image_citation():** Cite a fonte (citação) e os detalhes das imagens recuperadas da função `get_similar_image_from_query()`.
* **get_gemini_response():** Interage com um modelo Gemini para responder perguntas com base em uma combinação de entradas de texto e imagem.

In [ ]:
from utils.intro_multimodal_rag_utils import (
    get_similar_text_from_query,
    print_text_to_text_citation,
    get_similar_image_from_query,
    print_text_to_image_citation,
    get_gemini_response,
    display_images,
)

Antes de implementar um RAG multimodal, vamos dar um passo atrás e explorar o que você pode conseguir apenas com embeddings de texto ou imagem. Isso ajudará a estabelecer as bases para a implementação de um RAG multimodal, o que você fará na parte posterior do notebook. Você também pode usar esses elementos essenciais juntos para criar aplicativos para casos de uso multimodais para extrair informações significativas de documentos.

## Busca textual

Vamos começar a pesquisa com uma pergunta simples e ver se a pesquisa de texto simples usando incorporações de texto pode respondê-la. A resposta esperada é mostrar o valor do lucro líquido básico e diluído por ação do Google para diferentes tipos de ações.

In [ ]:
query = "Preciso de detalhes sobre o lucro líquido básico e diluído por ação Classe A, Classe B e Classe C do Google?"

### Fazendo *matching* na busca textual

In [ ]:
# Matching user text query with "chunk_embedding" to find relevant chunks.
matching_results_text = get_similar_text_from_query(
    query,
    text_metadata_df,
    column_name="text_embedding_chunk",
    top_n=3,
    chunk_text=True,
)

# Print the matched text citations
print_text_to_text_citation(matching_results_text, print_top=True, chunk_text=True)

Embora a resposta tenha sido devolvida, ela não retornou o valor relevante, que é o lucro líquido básico e diluído por ação. Isso ocorre porque as informações estão disponíveis apenas nas imagens e não na parte do texto do documento. Este é um dos exemplos de uma pesquisa de texto padrão (incluindo RAG de texto) que não retorna informações relevantes, embora as informações estejam presentes no documento.

**Importante:** criamos exemplos artesanais em nosso documento para simular casos do mundo real em que as informações geralmente estão incorporadas em gráficos, tabelas, gráficos e outros elementos baseados em imagens e não estão disponíveis como texto simples.

### Busque imagens a partir de buscas textuais

Como a pesquisa de texto simples não forneceu a resposta desejada e as informações podem ser representadas visualmente em uma tabela ou outro formato de imagem, você usará o recurso multimodal do modelo `Gemini Pro Vision`. O objetivo aqui também é encontrar uma imagem semelhante à consulta de texto. Você também pode listar as citações para verificar.

In [ ]:
matching_results_image = get_similar_image_from_query(
    text_metadata_df,
    image_metadata_df,
    query=query,
    column_name="text_embedding_from_image_description",  # Use image description text embedding
    image_emb=False,  # Use text embedding instead of image embedding
    top_n=3
)

# Markdown(print_text_to_image_citation(matching_results_image, print_top=True))
print("\n **** Resultado: ***** \n")

# Display the top matching image
display(matching_results_image[0]["image_object"])


A busca encontrou exatamente o que você procurava. Você queria os detalhes sobre o lucro líquido básico e diluído das ações Classe A, B e C do Google. Esta imagem se encaixa perfeitamente graças aos seus metadados descritivos usando Gemini.

Você também pode ver a citação e verificar o que Gemini extraiu como descrição para o resultado principal.

In [ ]:
## you can check the citations to probe further.
## check the "image description:" which is a description extracted through gemini which helped search our query.
Markdown(print_text_to_image_citation(matching_results_image, print_top=True))

## Busca por imagem

### Similaridade de imagens utilizando a busca por imagem

Imagine pesquisar imagens, mas em vez de digitar palavras, você usa uma imagem real como pista. Você tem uma tabela com números sobre o custo da receita de dois anos e deseja encontrar outras imagens semelhantes, no mesmo documento ou em vários documentos.

Pense nisso como pesquisar com um minimapa em vez de um endereço escrito. É uma maneira diferente de perguntar: “Mostre-me mais coisas como esta”. Então, em vez de digitar “tabela de custo de receita 2020 2021”, você mostra uma imagem dessa tabela e diz: “Encontre-me mais como isto”

Para fins de demonstração, encontraremos apenas imagens semelhantes que mostram o custo da receita ou valores semelhantes em um único documento abaixo. No entanto, você pode dimensionar esse padrão de design para corresponder (encontrar imagens relevantes) em vários documentos.

In [ ]:
# You can find a similar image as per the images you have in the metadata.
# In this case, you have a table (picked from the same document source) and you would like to find similar tables in the document.
image_query_path = "tac_table_revenue.png"

# Print a message indicating the input image
print("***Imagem enviada pelo usuário:***")

# Display the input image
Image.load_from_file(image_query_path)

Você espera encontrar tabelas (como imagens) semelhantes em termos de "Other/Total cost of revenues".

In [ ]:
# Search for Similar Images Based on Input Image and Image Embedding

matching_results_image = get_similar_image_from_query(
    text_metadata_df,
    image_metadata_df,
    query=query,  # Use query text for additional filtering (optional)
    column_name="mm_embedding_from_img_only",  # Use image embedding for similarity calculation
    image_emb=True,
    image_query_path=image_query_path,  # Use input image for similarity calculation
    top_n=3,  # Retrieve top 3 matching images
    embedding_size=1408,  # Use embedding size of 1408
)

print("\n **** Resultado: ***** \n")

# Display the Top Matching Image
display(
    matching_results_image[0]["image_object"]
)  # Display the top matching image object (Pillow Image)


O modelo encontrou uma imagem (tabela) de aparência semelhante, que fornece mais detalhes sobre diferentes receitas, despesas, receitas e mais alguns detalhes com base na imagem fornecida. Mais importante ainda, ambas as tabelas mostram números relacionados ao “custo da receita”.

Você também pode listar a citação para ver o que ela corresponde.

In [ ]:
# Display citation details for the top matching image
print_text_to_image_citation(matching_results_image, print_top=True)  # Print citation details for the top matching image

In [ ]:
# Check Other Matched Images (Optional)
# You can access the other two matched images using:

print("---------------Imagens encontradas------------------\n")
display_images(
    [
        matching_results_image[0]["img_path"],
        matching_results_image[1]["img_path"],
    ],
    resize_ratio = 0.8
)

A capacidade de identificar textos e imagens semelhantes com base na entrada do usuário, alimentada pelo Gemini e embeddings, constitui uma base crucial para o desenvolvimento de sistemas RAG multimodais, que você explorará na próxima seção.

### *Reasoning* comparativo

Imagine que você tem um gráfico que mostra o desempenho das ações Classe A do Google em comparação com outras coisas, como o S&P 500 ou outras empresas de tecnologia. Você quer saber o desempenho das ações da Classe C em comparação com esse gráfico. Em vez de apenas encontrar outra imagem semelhante, você pode pedir o Gemini para comparar as imagens relevantes e dizer em qual ação seria melhor para você investir. Gemini então explicaria o *reasoning* da sugestão.

O objetivo é enviar uma imagem de referência (ações Classe A do Google) e fazer perguntas simples de raciocínio como “Como isso se compara às ações Classe C?”. Você espera que os embeddings de imagens encontrem a imagem dos compartilhamentos Classe C e, em seguida, enviem a imagem de referência (compartilhamento Classe A) e a imagem semelhante (compartilhamento Classe C) para o Gemini e façam um raciocínio comparativo.

In [ ]:
matching_results_image_query_1 = get_similar_image_from_query(
    text_metadata_df,
    image_metadata_df,
    query="Mostre os gráficos que demonstram o retorno acumulativo total em 5 anos de Google Class A",
    column_name="text_embedding_from_image_description",  # Use image description text embedding # mm_embedding_from_img_only text_embedding_from_image_description
    image_emb=False,  # Use text embedding instead of image embedding
    top_n=3,
    embedding_size=1408,
)

In [ ]:
print("---------------Imagens encontradas------------------\n")
display_images(
    [
        matching_results_image_query_1[0]["img_path"],
        matching_results_image_query_1[1]["img_path"],
    ],
    resize_ratio=0.5,
)

In [ ]:
# image selected by the model to make the comparision based on user query
Image.load_from_file(image_selected_based_on_source_image[0]["img_path"])

In [ ]:
prompt = f""" Instruções: Compare as imagens e o texto extraído do Gemini fornecido como Contexto: para responder à pergunta:
Pense bem antes de responder à pergunta e defina as etapas necessárias para chegar à resposta em marcadores para facilitar a explicação.

Contexto:
Imagem_1: {matching_results_image_query_1[0]["image_object"]}
gemini_extracted_text_1: {matching_results_image_query_1[0]['image_description']}
Imagem_2: {matching_results_image_query_1[1]["image_object"]}
gemini_extracted_text_2: {matching_results_image_query_1[2]['image_description']}

Pergunta:
  - Principais conclusões das ações Classe A?
  - Quais são as diferenças críticas entre os gráficos das Ações Classe A?
  - Quais são as principais conclusões das ações Classe A em relação ao S&P 500?
  - Qual índice corresponde melhor ao desempenho das ações Classe A onde o Google ainda não faz parte? Explique o raciocínio.
  - Identifique os principais padrões gráficos em ambos os gráficos.
  - Qual índice corresponde melhor ao desempenho das ações Classe A onde o Google ainda não faz parte? Explique o raciocínio.
"""

# Generate Gemini response with streaming output
Markdown(
    get_gemini_response(
        model,
        model_input=[prompt],
        stream=True,
        generation_config=GenerationConfig(temperature=0.4, max_output_tokens=2048),
    )
)


<div class="alert alert-block alert-warning">
<b>⚠️ IMPORTANTE: Este não é um verdadeiro conselho de investimento e não deve ser levado a sério!! ⚠️</b>
</div>


## RAG multimodal

Vamos juntar tudo para implementar o RAG multimodal. Você usará todos os elementos explorados nas seções anteriores para implementar o RAG multimodal. Estas são as etapas:

* **Etapa 1:** O usuário faz uma consulta em formato de texto onde as informações esperadas estão disponíveis no documento e incorporadas em imagens e texto.
* **Etapa 2:** Encontre todos os trechos de texto das páginas dos documentos usando um método semelhante ao que você explorou em `Pesquisa de texto`.
* **Etapa 3:** Encontre todas as imagens semelhantes nas páginas com base na consulta do usuário correspondente a `image_description` usando um método idêntico ao que você explorou em `Image Search`.
* **Etapa 4:** Combine todos os textos e imagens semelhantes encontrados nas etapas 2 e 3 como `context_text` e `context_images`.
* **Etapa 5:** Com a ajuda do Gemini, podemos passar a consulta do usuário com o contexto de texto e imagem encontrado nas etapas 2 e 3. Você também pode adicionar uma instrução específica que o modelo deve lembrar ao responder à consulta do usuário.
* **Etapa 6:** Gemini produz a resposta e você pode listar as citações para verificar todos os textos e imagens relevantes usados para responder à consulta.

### Etapa 1: Requisição do usuário

In [ ]:
# this time we are not passing any images, but just a simple text query.

query = """Pergunta: Quais os melhores resultados, NASDAQ Composite ou ações Classe A do Google?
Qual seria melhor comprar e por quê?
Resposta: """

### Etapa 2: Busca de todos os *chunks* de texto relevantes

In [ ]:
# Retrieve relevant chunks of text based on the query
matching_results_chunks_data = get_similar_text_from_query(
    query,
    text_metadata_df,
    column_name="text_embedding_chunk",
    top_n=10,
    chunk_text=True,
)

### Etapa 3: Busca de todas as imagens relevantes

In [ ]:
# Get all relevant images based on user query
matching_results_image_fromdescription_data = get_similar_image_from_query(
    text_metadata_df,
    image_metadata_df,
    query=query,
    column_name="text_embedding_from_image_description",
    image_emb=False,
    top_n=3,
    embedding_size=1408,
)


### Step 4: Crie o `context_text` e o `context_images`

In [ ]:
# combine all the selected relevant text chunks
context_text = []
for key, value in matching_results_chunks_data.items():
    context_text.append(value["chunk_text"])
final_context_text = "\n".join(context_text)

# combine all the relevant images and their description generated by Gemini
context_images = []
for key, value in matching_results_image_fromdescription_data.items():
    context_images.extend(
        ["Imagem: ", value["image_object"], "Descrição: ", value["image_description"]]
    )


### Step 5: Passe o contexto ao Gemini

In [ ]:
instructions = """O contexto de extração de informações deve ser baseado no contexto de texto fornecido em "text_context" e no contexto de imagem fornecido em "image_context" junto com sua legenda: \n
Baseie sua resposta em "text_context" e "image_context". Não utilize números ou porcentagens que não estejam presentes no "image_context".
Utilize somente informações presentes no contexto, não utilize nenhuma outra informação. Considere apenas os anos citados no contexto.
Não inclua nenhum retorno total cumulativo na resposta. Contexto:
"""

final_prompt = [
    query,
    instructions,
    "text_context:",
    "\n".join(context_text),
    "image_context:",
]
final_prompt.extend(context_images)

In [ ]:
Markdown(get_gemini_response(model, model_input = final_prompt,stream=True))

In [ ]:
print("---------------Imagens encontradas------------------\n")
display_images(
    [
        matching_results_image_fromdescription_data[0]["img_path"],
        matching_results_image_fromdescription_data[1]["img_path"],
    ],
    resize_ratio = 0.8
)

### Etapa 6: Liste citações e referências

In [ ]:
# Image citations. You can check how Gemini generated metadata helped in grounding the answer.

print_text_to_image_citation(matching_results_image_fromdescription_data, print_top=False)

In [ ]:
# Text citations

print_text_to_text_citation(
    matching_results_chunks_data,
    print_top=False,
    chunk_text=True,
)


## Conclusões

Embora o RAG multimodal possa ser bastante poderoso, observe que ele pode enfrentar algumas limitações:

* **Dependência de dados:** precisa de texto e recursos visuais de alta qualidade.
* **Computacionalmente exigente:** O processamento de dados multimodais consome muitos recursos.
* **Específico do domínio:** Modelos treinados em dados gerais podem não brilhar em áreas especializadas como medicina.
* **Opacidade dos modelos:** Compreender como esses modelos funcionam pode ser complicado, prejudicando a confiança e a adoção.

Apesar destes desafios, o RAG multimodal representa um passo significativo em direção a sistemas de busca e recuperação que podem lidar com dados multimodais diversos.